# Attention e modello Transformer, capitolo 16 di *Hands-On Machine Learning... (2019)*

**Elementi trattati in capitoli precedenti nel libro e citati o utilizzati nel capitlolo 16:**

* Chapter 4, dot product
* Chapter 4, gradient descent
* Chapter 13, custom preprocessing layer
* Chapter 13, categorical input features
* Chapter 13, TensorFlow datasets
* Chapter 13, look-up table
* Chapter 13, TF Transform
* Chapter 13, Embedding layer
* Chapter 15, RNNs
* Chapter 15, time-distributed Dense layer




**Esercizi di fine capitolo**

1. Quali sono i pro e i contro di utilizzare una RNN *stateful* rispetto a una RNN *stateless*?
  
  Un modello stateless a ogni iterazione parte con uno stato *azzerato* che viene aggiornato ad ogni step. All'ultimo step, lo stato non viene conservato perchè ha terminato il suo utilizzo, e alla successiva iterazione si riparte da zero.

  In un modello stateful invece lo stato all'ultimo step di una iterazione viene conservato come punto di partenza per l'iterazione successiva. Un modello stateful può quindi valutare pattern più lunghi; tuttavia un modello di questo tipo richiede più lavoro di pre-processing in quanto ogni sequenza in un batch deve cominiciare esattamente da dove finisce la precedente.

2. Perchè per l'*automatic translation* si utilizzano RNN di tipo Encoder-Decoder al posto di semplici RNN *sequence-to-sequence*

  Vengono preferiti modelli Encoder-Decoder in quanto possono valutare una frase nella sua interezza. I modelli sequence-to-sequence invece traducono parola per parola, e questo tipo di approccio porta a scarsi risultati.

3. In che modo si possono gestire sequenze di input di lunghezza variabile? E di output?

  Per quanto riguarda le sequenze di input, si possono usare dei token di padding, a patto che venga istruito un layer di masking per far sì che la rete neurale li ignori nel calcolo dell'errore. In output invece si può usare un token di *end-of-sequence*; anche in questo caso bisogna istruire la rete perchè ignori gli elementi della sequenza che vengono dopo tale token.

4. Cos'è la *beam search*, e quando si utilizza? Con quale strumento si può implementare?

  Beam search è una soluzione per fare in modo che il modello possa ad ogni step tenere conto delle $k$ sequenze che hanno maggiore probabilità di essere corrette. Ad ogni step ogni sequenza si allunga di una parola, e alla fine viene selezionata la sequenza, vale a dire la frase, con la maggiore probabilità.

5. Cos'è un meccanismo di attenzione? Qual è la sua utilità?

  Si tratta di un meccanismo che permette al modello di concentrarsi a ogni step su un particolare elemento della sequenza. Per esempio, prendendo in analisi un traduttore inglese-francese, allo step temporale in cui il decoder deve produrre in output la parola *lait*, concentrerà la sua attenzione sulla parola *milk* in input. Questo permette di abbreviare notevolmente il percorso di ogni parola in input fino alla sua traduzione, migliorando le prestazioni delle RNN per il natural language processing.

6. Qual è il layer più importante nell'architettura *Transformer*? Qual è il suo scopo?

  Il layer più importante nell'archittettura Transformer è il Multi-Head Attention layer. Esso utilizza delle rappresentazioni in vettori dei concetti sintattici (es. soggetto, verbo, ...) delle varie parole per trovare una similarità fra la parola input e la parola target. Per calcolare questa similarità utilizza un tipo di prodotto scalare regolarizzato chiamato *Scaled Dot-Product Attention*.

7. Quando si utilizza il *sampled softmax*?

  Il sampled softmax si utilizza in NMT quando si ha un vocabolario molto ampio e quindi calcolare la probabilità di ogni singola parola sarebbe troppo dispendioso. Per ovviare a questo problema, la probabilità si calcola solo sulla parola corretta e su un campione (un sample) di parole errate prese casualmente.

10. Esegui il tutorial di TensorFlow sul NMT con Attention

  Il tutorial si trova a questo [link](https://www.tensorflow.org/tutorials/text/nmt_with_attention).

  TensorFlow ha realizzato anche un [tutorial simile per i Transformer](https://www.tensorflow.org/tutorials/text/transformer). 

9. Addestra un modello Encoder-Decoder per convertire delle date da un formato all'altro (es. da"April 22, 2019" a "2019-04-22")

(Il codice è preso dalle soluzioni incluse nel libro)

**Creazione del dataset**

In [ ]:
import numpy as np
from datetime import date

MONTHS = ["January", "February", "March", "April", "May", "June",
          "July", "August", "September", "October", "November", "December"]

def random_dates(n_dates):
    min_date = date(1000, 1, 1).toordinal()
    max_date = date(9999, 12, 31).toordinal()

    ordinals = np.random.randint(max_date - min_date, size=n_dates) + min_date
    dates = [date.fromordinal(ordinal) for ordinal in ordinals]

    x = [MONTHS[dt.month - 1] + " " + dt.strftime("%d, %Y") for dt in dates]
    y = [dt.isoformat() for dt in dates]
    return x, y

In [ ]:
x, y = random_dates(1)
print(f"Input sequence: {x}, output sequence: {y}")

Input sequence: ['January 01, 6478'], output sequence: ['6478-01-01']


* Tutti i possibili caratteri in input

In [ ]:
INPUT_CHARS = "".join(sorted(set("".join(MONTHS) + "0123456789, ")))
INPUT_CHARS

' ,0123456789ADFJMNOSabceghilmnoprstuvy'

* Tutti i possibili caratteri in output

In [ ]:
OUTPUT_CHARS = "0123456789-"

* Funzione per convertire una stringa in una lista di ID

In [ ]:
def date_str_to_ids(date_str, chars=INPUT_CHARS):
    return [chars.index(c) for c in date_str]

[15, 20, 29, 35, 20, 32, 37, 0, 2, 3, 1, 0, 8, 6, 9, 10]

In [ ]:
date_str_to_ids(x[0])

[15, 20, 29, 35, 20, 32, 37, 0, 2, 3, 1, 0, 8, 6, 9, 10]

In [ ]:
date_str_to_ids(y[0], OUTPUT_CHARS)

[6, 4, 7, 8, 10, 0, 1, 10, 0, 1]

In [ ]:
import tensorflow as tf

def prepare_date_strs(date_strs, chars=INPUT_CHARS):
    X_ids = [date_str_to_ids(dt, chars) for dt in date_strs]
    X = tf.ragged.constant(X_ids, ragged_rank=1)
    return (X + 1).to_tensor() # using 0 as the padding token ID

In [ ]:
def create_dataset(n_dates):
    x, y = random_dates(n_dates)
    return prepare_date_strs(x, INPUT_CHARS), prepare_date_strs(y, OUTPUT_CHARS)

np.random.seed(42)

X_train, Y_train = create_dataset(10000)
X_valid, Y_valid = create_dataset(2000)
X_test, Y_test = create_dataset(2000)

**Creazione del modello**


L'encoder è formato da un Embedding layer seguito da un Long short-term memory layer. L'output dell'encoder, un vettore, viene passato al decoder, che è formato da un Long short-term memory layer e un Dense layer con attivazione softmax. L'output del decoder è un vettore contenente le probabilità per tutti i possibili caratteri di output (`OUTPUT_CHARS`).




In [ ]:
from tensorflow import keras

embedding_size = 32
max_output_length = Y_train.shape[1]

np.random.seed(42)
tf.random.set_seed(42)

encoder = keras.models.Sequential([
    keras.layers.Embedding(input_dim=len(INPUT_CHARS) + 1,
                           output_dim=embedding_size,
                           input_shape=[None]),
    keras.layers.LSTM(128)
])

decoder = keras.models.Sequential([
    keras.layers.LSTM(128, return_sequences=True),
    keras.layers.Dense(len(OUTPUT_CHARS) + 1, activation="softmax")
])

model = keras.models.Sequential([
    encoder,
    keras.layers.RepeatVector(max_output_length),
    decoder
])

optimizer = keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

* Addestramento

In [ ]:
history = model.fit(X_train, Y_train, epochs=20, validation_data=(X_valid, Y_valid))

Epoch 1/20
313/313 [==============================] - 18s 45ms/step - loss: 2.0799 - accuracy: 0.2643 - val_loss: 1.3714 - val_accuracy: 0.4938
Epoch 2/20
313/313 [==============================] - 13s 42ms/step - loss: 1.2944 - accuracy: 0.5237 - val_loss: 1.0406 - val_accuracy: 0.6212
Epoch 3/20
313/313 [==============================] - 13s 42ms/step - loss: 1.0312 - accuracy: 0.6311 - val_loss: 0.8331 - val_accuracy: 0.6978
Epoch 4/20
313/313 [==============================] - 13s 42ms/step - loss: 0.8362 - accuracy: 0.6997 - val_loss: 0.6144 - val_accuracy: 0.7645
Epoch 5/20
313/313 [==============================] - 13s 42ms/step - loss: 0.5807 - accuracy: 0.7782 - val_loss: 0.4772 - val_accuracy: 0.8145
Epoch 6/20
313/313 [==============================] - 13s 42ms/step - loss: 0.4087 - accuracy: 0.8422 - val_loss: 0.3090 - val_accuracy: 0.8826
Epoch 7/20
313/313 [==============================] - 13s 41ms/step - loss: 0.3920 - accuracy: 0.8630 - val_loss: 0.3521 - val_accuracy:

* Per vedere le date in formato stringa occorre convertire gli ID dei caratteri

In [ ]:
def ids_to_date_strs(ids, chars=OUTPUT_CHARS):
    return ["".join([("?" + chars)[index] for index in sequence])
            for sequence in ids]

In [ ]:
X_new = prepare_date_strs(["September 17, 2009", "July 14, 1789"])
# ids = model.predict_classes(X_new) deprecato
ids = np.argmax(model.predict(X_new), axis=-1)
for date_str in ids_to_date_strs(ids):
    print(date_str)

2009-09-17
1789-07-14


In [ ]:
X_new = prepare_date_strs(["May 02, 2020", "July 14, 1789"])
ids = np.argmax(model.predict(X_new), axis=-1)
for date_str in ids_to_date_strs(ids):
    print(date_str)

2020-03-02
1789-01-14


* Dopo 20 epoche di training, il modello raggiunge una precisione del 100%. Tuttavia, con input di lunghezza minori di 18 caratteri, non è affidabile. Per ovviare a questo problema, bisogna fornire in ingresso date lunghe 18 caratteri, aggiungendo un eventuale padding finale:

In [ ]:
max_input_length = X_train.shape[1]

def prepare_date_strs_padded(date_strs):
    X = prepare_date_strs(date_strs)
    if X.shape[1] < max_input_length:
        X = tf.pad(X, [[0, 0], [0, max_input_length - X.shape[1]]])
    return X

def convert_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    ids = np.argmax(model.predict(X), axis=-1)
    return ids_to_date_strs(ids)

In [ ]:
convert_date_strs(["May 02, 2020", "July 14, 1789"])

['2020-05-02', '1789-07-14']

10. Utilizzare uno dei più recenti modelli di NLP (BERT, GPT, ...) per generare del testo shakespeariano.

Per implementare uno dei modelli più avanzati citati nel libro, si può fare uso della libreria open source *Transformers* di Hugging Face.

In [2]:
!pip install transformers

     |████████████████████████████████| 2.1MB 8.9MB/s 
     |████████████████████████████████| 3.3MB 44.3MB/s 
     |████████████████████████████████| 901kB 45.2MB/s 


In questo caso, viene implementato il modello GPT di OpenAI in una configurazione già addestrata.

In [21]:
from transformers import TFOpenAIGPTLMHeadModel

model = TFOpenAIGPTLMHeadModel.from_pretrained("openai-gpt")

All model checkpoint layers were used when initializing TFOpenAIGPTLMHeadModel.

All the layers of TFOpenAIGPTLMHeadModel were initialized from the model checkpoint at openai-gpt.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFOpenAIGPTLMHeadModel for predictions without further training.


Come tokenizer, di default verrà utilizzato il `BasicTokenizer` del modello BERT. La documentazione riporta che questo tokenizer dovrebbe funzionare bene per la maggior parte degli usi, altrimenti sarà possibile sostituirlo con quelli utilizzati nel *paper* originale (SpaCy e ftfy).

In [46]:
from transformers import OpenAIGPTTokenizer

tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


Test del tokenizer:

In [47]:
prompt_text = "This royal throne of kings, this sceptred isle"
encoded_prompt = tokenizer.encode(prompt_text,
                                  add_special_tokens=False,
                                  return_tensors="tf")
encoded_prompt

<tf.Tensor: shape=(1, 10), dtype=int32, numpy=
array([[  616,  5751,  6404,   498,  9606,   240,   616, 26271,  7428,
        16187]], dtype=int32)>

Ora possiamo utilizzare il modello. L'idea è di partire da un testo in input, e da quello generare $n$ sequenze di $k$ caratteri. Il tuning degli iperparametri è spiegato in [questo articolo](https://huggingface.co/blog/how-to-generate) sul blog di Hugging Face

In [48]:
num_sequences = 3
length = 50

generated_sequences = model.generate(
    input_ids=encoded_prompt, # sequenza di input
    do_sample=True, # attivazione del sampling
    max_length=length + len(encoded_prompt[0]), # lunghezza dell'output (length + input)
    temperature=1.0, # la temperatura definisce quanto il modello è "sensibile" ai candidati con bassa proabilità
    top_k=0, # disattivazione del top_k sampling
    top_p=0.9, # nucleus sampling
    repetition_penalty=1.0, # penalizzazione di parole già utilizzate
    num_return_sequences=num_sequences, # numero di sequenze da generare
)

generated_sequences

<tf.Tensor: shape=(3, 60), dtype=int32, numpy=
array([[  616,  5751,  6404,   498,  9606,   240,   616, 26271,  7428,
        16187,   544,   246,  2592,   488,  3128,  1082,   239,   616,
          544,   895,   512,   635,   538,  1085,   246,  1082,   485,
         2830,   551,   239,   616,   566,   635,   580,  3072,   267,
          256, 40477,   256,   249,   993,   538,   587,   525,   267,
          481,   618,   812,   848,   793,   669,   487,  7909,   239,
          256, 40477,   256,   674,   512,  1259],
       [  616,  5751,  6404,   498,  9606,   240,   616, 26271,  7428,
        16187,   240,   645,   507,   641,   595,   562,   481,  6308,
         4189,   877,   498,   481,  3573,   240,   636,   604,  1256,
          725,  4837,   485,   481,  1657,  6903,   239,  5476,   507,
          509,   525,   669,   481,   966, 14154,   485,  3013,   481,
        15577,   240,   488,   481,  2565,   498,   481, 32661,   509,
          885,   525,   655,   994,   580,   664],

L'output è in formato tokenizzato e va convertito al formato testuale

In [49]:
for sequence in generated_sequences:
    text = tokenizer.decode(sequence, clean_up_tokenization_spaces=True)
    print(text)
    print("." * 80)

this royal throne of kings, this sceptred isle is a serious and powerful place. this is why you couldn't find a place to hide out. this one could be dangerous!'
'i can't do that! the king will come here when he wishes.'
'then you must
................................................................................
this royal throne of kings, this sceptred isle, if it were not for the fierce governance of the island, would have done more harm to the whole realm. thus it was that when the need arose to protect the succession, and the promise of the scepter was made that there should be no
................................................................................
this royal throne of kings, this sceptred isle was set up a very peaceful thing in which you couldn't worry about tax laws, not even in the age of the kings. " 
 " but, " garion said, " isn't that supposed to mean that i'm to be king? "
................................................................................
